# Numerical Analysis Project

Team: 아프지마

Member
* 20181628 성창훈
* 20181635 신명철

written by 성창훈, 신명철

# Google Colab

원할한 학습을 위해서 Google Colab의 GPU를 이용해서 학습한다.

Google Colab에서는 Tesla K80과 Tesla T4 두 종류의 GPU를 제공한다.

이 노트에서는 K80보다 성능이 더 좋은 Tesla T4를 이용한다.

In [1]:
!nvidia-smi

Mon May 24 15:57:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Packages

대다수의 Web-Based notebook 서비스들(Google Colab, Kaggle, Paperspace...)은 docker를 이용해 일관된 환경을 제공한다.

환경의 업데이트는 패키지의 업데이트보다 보수적으로 이루어지므로 보통 버전이 낮다.

이때 일부 최신 패키지는 높은 버전의 pip 버전을 요하므로, 반드시 패키지 관리 도구를 최신 버전으로 업데이트하여 사용한다.

업데이트 후에는 kernel을 다시 시작한다. (초기화가 아님에 유의한다)

In [2]:
!pip install --upgrade pip
!pip install --upgrade setuptools

런타임 다시 시작

In [1]:
!pip install auto-sklearn

# Auto-Sklearn

가장 유명한 파이썬 데이터 분석 패키지 중 하나인 Sklearn의 AutoML 버전이다.

AutoSklearn 2.0의 Classifier을 통해 이미지를 분류하는 모델을 자동으로 탐색하고(NAS), 하이퍼파라메터를 최적화한다(HPO).

In [7]:
import sklearn.datasets
import sklearn.metrics

from autosklearn.experimental.askl2 import AutoSklearn2Classifier

import tensorflow as tf
import numpy as np

## Data Loading

Kaggle의 Leaf Rice Disease Dataset을 이용한다.

모든 노트에서 데이터 처리하는 것이 번거로우므로, ndarray화 시킨 이미지 데이터셋을 구글 드라이브에 업로드한 후, 각 노트에서 불러와 사용했다.

dataset = imgset(small_rgb).npy.npz

* `dataset['x'].shape : (240, 360, 480, 3)` -> Numpy ndarray image data
* `dataset['y'].shape : (240, )` -> labels composed of 0, 1, 2


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
dataset = np.load('/content/drive/MyDrive/imgset(small_rgb).npy.npz')
data = dataset['x']
labels = dataset['y']

In [24]:
d1, d2, d3, d4 = data.shape
data_reshaped = data.reshape(d1, d2 * d3 * d4)

In [25]:
ion.train_test_split(data_reshaped, labels, test_size=0.2)

## Build a classifier and Fit

AutoSklearn이 classification에 적합한 모델을 탐색한다.

코드가 간단한 것이 AutoML의 큰 장점인데, 모델 설계와 학습을 단 두 줄에 끝냈다.

시간과 자원을 고려해 최대 탐색 시간은 30분으로 제한했다.

In [30]:
automl = AutoSklearn2Classifier(
    time_left_for_this_task=1800,
    n_jobs=-1,
)
automl.fit(X_train, y_train, dataset_name='leaf_disease')

Using Backup selector


/usr/local/lib/python3.7/dist-packages/distributed/node.py:164: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43839 instead
  expected, actual


AutoSklearn2Classifier(n_jobs=-1, per_run_time_limit=360,
                       time_left_for_this_task=1800)

## Print the pipelines constructed by auto-sklearn

show_models() 메소드를 통해 fit()을 진행하면서 AutoSklearn이 찾아낸 모델들을 볼 수 있다.

파이프라인 내부를 보면 모델 구조나 데이터 전처리, 경사하강 기법, 학습 속도 등을 찾아낸 것을 알 수 있다.



In [31]:
print(automl.show_models())

[(0.300000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'extra_trees', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessing:numerical_transformer:imputation:strategy': 'median', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'none', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:extra_trees:bootstrap': 'False', 'classifier:extra_trees:criterion': 'gini', 'classifier:extra_trees:max_depth': 'None', 'classifier:extra_trees:max_features': 0.5389212988218297, 'classifier:extra_trees:max_leaf_nodes': 'None', 'classifier:extra_trees:min_impurity_decrease': 0.0, 'classifier:extra_trees:min_samples_leaf': 1, 'classifier:extra_trees:min_samples_split': 2, 'classifier:extra_trees:min_weight_fraction_leaf': 0.0},
dataset_properties={
  'task': 2,
  'sparse

## Get accuracy of the final ensemble model

앙상블 기법으로 완성된 최종 모델의 성능을 테스트한다.

30분동안 학습한 결과 test_accuracy 83%를 달성했다.



In [32]:
predictions = automl.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

Accuracy score: 0.8333333333333334
